<a href="https://colab.research.google.com/github/MWaser/BurstIQ/blob/main/CT_Embed.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# CT_Studies Vector Embedding

# Grab data from Snowflake

There are currently 144K clinical trials that are either recruiting or awaiting recruiting.  
These have been placed in the C2_CTRIAL.CLINICAL_TRIALS.CT_SEARCH data set.

The embedding calculation takes about 0.1 seconds so we will run in batches of 20K (a little over 30 minutes per)

Note: Yes, I *did* run it 8 times changing the startrow and save file name.<br /> &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; Yes, it can be refactored to do all iterations in a loop and I'll probably do so when doing larger datasets.

In [ ]:
!pip install snowflake-connector-python
import snowflake.connector
import pandas as pd

In [61]:
startrow = 120001
cnx = snowflake.connector.connect(user='waserma', password='snowH34d',
        account='csi_eval.us-east-1-gov.aws',
        warehouse='WAREHOUSE1' )
curs = cnx.cursor()
# query = 'SELECT * FROM C2_CTRIAL.CLINICAL_TRIALS.CT_SEARCH WHERE ROWNUM >= ' + str(startrow) + ' AND ROWNUM < ' + str(startrow + 20000) + ' ORDER BY ROWNUM'
query = 'SELECT * FROM C2_CTRIAL.CLINICAL_TRIALS.CT_SEARCH WHERE ROWNUM >= ' + str(startrow) + ' ORDER BY ROWNUM'
snowData = curs.execute(query).fetchall()
column_names = [i[0] for i in curs.description]
curs.close()
tempDF = pd.DataFrame(snowData, columns=column_names)
tempDF.head(7)

,NCT_ID,BRIEF_TITLE,OFFICIAL_TITLE,DESCRIPTION,EMBED,ROWNUM
0,NCT05846269,A Resiliency and TIC Intervention for PMHNP Sp...,A Resiliency and Trauma-Informed Care Interven...,Students in the Doctor of Nursing Practice (DN...,,120001
1,NCT05846282,Students Rising Above: Offsetting the Health a...,Project STRIVE (STudents RIsing Above) - Offse...,The STRIVE and SOAR interventions will be tria...,,120002
2,NCT05846308,The Effectiveness of Group Interpersonal Synch...,The Effectiveness of Group Interpersonal Synch...,Background: Few autistic adults are able to in...,,120003
3,NCT05846321,Registry for Vascular Trauma and Follow-up Exa...,Registry for Vascular Trauma and Follow-up Exa...,None,,120004
4,NCT05846334,mHealth Intervention to Reduce Perceived Stres...,A mHealth Intervention to Reduce Perceived Str...,"Chronic stress occurs over an extended period,...",,120005
5,NCT05846347,Phase I Clinical Study of GC012F Injection in ...,Phase I Clinical Study of GC012F Injection in ...,Systemic lupus erythematosus (SLE) is a kind o...,,120006
6,NCT05846373,Effect of Nicotinic Acid as Add on Therapy in ...,Effect of Nicotinic Acid as add-on Therapy in ...,Migraine is a common episodic neurological dis...,,120007


# Add Embeddings

In [ ]:
!pip install -U sentence-transformers
from sentence_transformers import SentenceTransformer
model = SentenceTransformer('all-MiniLM-L6-v2')

In [62]:
tempDF['BRIEF_TITLE'] = tempDF['BRIEF_TITLE'].astype('str')
tempDF['OFFICIAL_TITLE'] = tempDF['OFFICIAL_TITLE'].astype('str')
tempDF['DESCRIPTION'] = tempDF['DESCRIPTION'].astype('str')

tempDF['EMBED'] = tempDF.apply(lambda row: model.encode(row['BRIEF_TITLE'] + row['OFFICIAL_TITLE'] + row['DESCRIPTION']), axis=1)
tempDF.head(7)

,NCT_ID,BRIEF_TITLE,OFFICIAL_TITLE,DESCRIPTION,EMBED,ROWNUM
0,NCT05846269,A Resiliency and TIC Intervention for PMHNP Sp...,A Resiliency and Trauma-Informed Care Interven...,Students in the Doctor of Nursing Practice (DN...,"[-0.01428679, -0.017538037, -0.009010618, 0.08...",120001
1,NCT05846282,Students Rising Above: Offsetting the Health a...,Project STRIVE (STudents RIsing Above) - Offse...,The STRIVE and SOAR interventions will be tria...,"[0.02048702, 0.055303406, -0.032865282, 0.0257...",120002
2,NCT05846308,The Effectiveness of Group Interpersonal Synch...,The Effectiveness of Group Interpersonal Synch...,Background: Few autistic adults are able to in...,"[-0.027654996, -0.022475084, 0.011510374, 0.02...",120003
3,NCT05846321,Registry for Vascular Trauma and Follow-up Exa...,Registry for Vascular Trauma and Follow-up Exa...,None,"[0.020756023, 0.018385772, 0.0030696527, 0.049...",120004
4,NCT05846334,mHealth Intervention to Reduce Perceived Stres...,A mHealth Intervention to Reduce Perceived Str...,"Chronic stress occurs over an extended period,...","[0.01165793, -0.027450496, -0.026125062, 0.077...",120005
5,NCT05846347,Phase I Clinical Study of GC012F Injection in ...,Phase I Clinical Study of GC012F Injection in ...,Systemic lupus erythematosus (SLE) is a kind o...,"[-0.10941171, -0.042033292, 0.06609217, 0.0052...",120006
6,NCT05846373,Effect of Nicotinic Acid as Add on Therapy in ...,Effect of Nicotinic Acid as add-on Therapy in ...,Migraine is a common episodic neurological dis...,"[-0.03430126, -0.06378337, 0.0075286888, 0.075...",120007


# Save to file on Google Drive


In [ ]:
from google.colab import drive
drive.mount('/drive')

In [63]:
tempDF.to_csv('/drive/My Drive/embed12.csv')